<a href="https://colab.research.google.com/github/p16111121/fcc_sms_text_classification_test/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_dataset = pd.read_csv(train_file_path, sep='\t', names=['type', 'msg'],header=None)
test_dataset = pd.read_csv(test_file_path, sep='\t', names=['type', 'msg'],header=None)

In [ ]:
train_dataset.head()

In [ ]:
train_label = train_dataset['type']
test_label = test_dataset['type']
train_label = train_label.map({"ham": 0, "spam": 1})
test_label = test_label.map({"ham": 0, "spam": 1})

train_data = train_dataset.drop('type',axis = 1)
test_data = test_dataset.drop('type',axis = 1)

In [ ]:
# prompt: create a model that takes a message string as an argument and returns a binary

from tensorflow.keras.layers import TextVectorization

MAX_TOKENS = 1000
OUTPUT_SEQUENCE_LENGTH = 100

vectorizer = TextVectorization(max_tokens=MAX_TOKENS,
                               output_sequence_length=OUTPUT_SEQUENCE_LENGTH)
vectorizer.adapt(train_data['msg'])

def vectorize_text(text):
  text = tf.expand_dims(text, -1)
  return vectorizer(text)

train_data = vectorize_text(train_data['msg'])
test_data = vectorize_text(test_data['msg'])

model = tf.keras.models.Sequential([
  tf.keras.layers.Embedding(MAX_TOKENS + 1, 32),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(train_data, train_label, epochs=10)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    processed_message = vectorize_text(tf.constant([pred_text]))
    type_prediction = model.predict(processed_message)[0][0]

    prediction = [type_prediction, "ham" if int(round(type_prediction)) == 0 else "spam"]
    return (prediction)

#def predict_message(message):
#  processed_message = vectorize_text(tf.constant([message]))
#  prediction = model.predict(processed_message)[0][0]
#  return int(round(prediction))

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
